## Check GPU and Tensorflow version
If "Found GPU at: / device: GPU: 0" is displayed, the GPU is ready to use.*italicized text*

In [ ]:
# Install python and training dependencies
!apt-get update -y
!sudo apt-get install python3.11 python3.11-distutils
!ln -sf /usr/bin/python3.11 /usr/bin/python3

!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11
!update-alternatives --config python3

!apt-get install python3-pip
!python3 -m pip install --upgrade pip --user

!pip3 uninstall keras-vis
!pip3 install git+https://github.com/sctse999/keras-vis

!pip3 install moviepy

!pip3 install ipykernel

!pip3 install imageio-ffmpeg

In [ ]:
# OPTIONAL Check installed tensorflow and python version
!pip show tensorflow
!python3 --version

In [ ]:
# Download and install DonkeyCar
!git clone https://github.com/autorope/donkeycar
%cd /content/donkeycar
!git checkout main
!pip3 install -e .[pc]

In [ ]:
# Create car
!donkey createcar --path /content/mycar

In [ ]:
# Change config in the following array of key-value pairs. KV Pairs will be changed in mycar/myconfig.py.
config=[("EARLY_STOP_PATIENCE","8"),("CREATE_TF_LITE","True"),("AUGMENTATIONS","[]"),("BATCH_SIZE","64"),
        ("MAX_EPOCHS","60"),("OPTIMIZER","sgd"),""]

def update_config(file_path, key_value_pairs):
    # Read the existing content from the file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Create a dictionary from the key-value pairs for quick lookup
    updates = {key: f"{key}={value}\n" for key, value in key_value_pairs}

    # Replace or uncomment lines containing the keys
    new_keys = updates.copy()
    commented_values = {}
    for i, line in enumerate(lines):
        original_line = line.strip()
        is_commented = original_line.startswith('#')
        uncommented_line = original_line.lstrip('#').strip()
        key_part = uncommented_line.split('=')[0].strip()

        if key_part in updates:
            # Capture the value from commented lines for possible replacement
            if is_commented:
                commented_values[key_part] = uncommented_line.split('=')[1].strip()
            # Replace or uncomment the line based on its status
            new_value = commented_values.get(key_part, updates[key_part].split('=')[1]).strip()

            lines[i] = f"{key_part}={new_value}\n"
            del new_keys[key_part]
    # Append any new keys that were not found in the file
    lines.extend(new_keys.values())

    # Write the modified content back to the file
    with open(file_path, 'w') as file:
        file.writelines(lines)

# Example usage
update_config('/content/mycar/myconfig.py', config)


In [ ]:
# Download zips with data, follow that zip structure
# /
# - images/
# - catalog_0.catalog
# - catalog_0.catalog_manifest
# ...

%cd /content/mycar/
!mkdir /content/mycar/data

file_names = []
file_paths = []

import requests

# URL of the file you want to download. If you put multiple files, you might have to change later following commands. But this and following
# cells can process multiple zip files. Recommended free file host is mediafire, because gdrive doesn't allow direct download links above a certain size
urls = ['<Put direct download link 1 here>']
counter = 0
# Send a GET request to the URL
for url in urls:
  response = requests.get(url)
  # Ensure the request was successful
  if response.status_code == 200:
    file_name = f"data-{counter}.zip"
    counter += 1
    file_names.append(file_name)
    with open(f'/content/{file_name}', 'wb') as f:
        # Write the content of the response to the file
        file_paths.append(f'/content/{file_name}')
        f.write(response.content)
        print("File downloaded successfully!")
  else:
    print("Failed to retrieve the file. Status code:", response.status_code)

In [ ]:
# Unzip downloaded files. Unzipped tubs will be in /content/data-0, /content/data-1, ...

import os
import shutil
from google.colab import files

if(os.path.exists(f"/content/mycar/data/{file_name}")):
   os.remove(f"/content/mycar/data/{file_name}")
file_path = "/content/mycar/data/" + file_name

! rm -r /content/mycar/data/*

import os

import zipfile

def unpack_zip(file_path, extract_to):
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
        print(f"Extracted all files in {file_path} to {extract_to}")
def unpack_file(file_path, extract_to):
    _, file_ext = os.path.splitext(file_path)

    if file_ext in ['.zip']:
        unpack_zip(file_path, extract_to)
    else:
        print(f"Unsupported file format: {file_ext}")
def safe_unpack_file(file_path, extract_to):
    try:
        print(f"unpacking: " + file_path)
        unpack_file(file_path, extract_to)
    except Exception as e:
        print(f"An error occurred: {e}")

print("-----------------------------------------------------")
print(f"Uploaded {file_path}")
print("-----------------------------------------------------")
print(f"found {len(file_names)} zips to unpack")
target_directory = "/content/mycar/data/"
src_directory = "/content/"
for filename in file_names:
    if os.path.isfile(os.path.join(target_directory, filename)):  # Check if it is a file
        # Extract the file name without the extension
        file_base = os.path.splitext(filename)[0]
        # Create a path for the new subfolder
        new_folder_path = os.path.join(target_directory, file_base)
for file_name in file_names:
  safe_unpack_file(src_directory + file_name, (target_directory + file_name)[:-len('.zip')])

In [ ]:
# OPTIONAL Only needed for data collected by Donkeycar version <= 4.5.x. 
# Prepare data by replacing "steering" with "user/angle" and "throttle" with "user/throttle"

import os
import re

def search_replace(directory, search_text, replace_text, file_prefixes=None):
    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            # Check if the filename starts with any of the specified prefixes
            if file_prefixes and not any(filename.startswith(prefix) for prefix in file_prefixes):
                continue  # Skip files that do not start with any of the specified prefixes

            filepath = os.path.join(dirpath, filename)
            # Read the content of the file
            with open(filepath, 'r', encoding='utf-8') as file:
                filedata = file.read()

            # Replace the target string
            filedata_new = re.sub(search_text, replace_text, filedata, flags=re.IGNORECASE)

            # Write the file out again if there are changes
            if filedata != filedata_new:
                with open(filepath, 'w', encoding='utf-8') as file:
                    file.write(filedata_new)
                print(f"Updated {filepath}")

search_text = ""
replace_directory = "/content/mycar/data"

search_replace(replace_directory, "steering", "user/angle", ["catalog_", "manifest"])
search_replace(replace_directory, "throttle", "user/throttle", "catalog_")

%cd /content/mycar/data/
!echo Found images:
!find . -type f -name "*.jpg" | wc -l



In [ ]:
# OPTIONAL generate a movie from given tub data
%cd /content/mycar
!donkey makemovie --tub ./data --out tub.mp4 --start 1 --end 2000

In [ ]:
# OPTIONAL Download result video
from google.colab import files
files.download("tub.mp4")

In [ ]:
# Starts training!

%cd /content/mycar/

model_name = "mypilot"

! donkey train --tub ./data/data-0 --model ./models/mypilot.h5

In [ ]:
# OPTIONAL Checks if an according tflite model could be found. (h5 and tflite will be generated, but documentation recommends
# using the tflite model)
!ls -alh /content/mycar/models/{model_name}.tflite

In [ ]:
# Show loss curve
%cd /content/mycar/models

import glob
file = glob.glob(f"{model_name}.png")

from IPython.display import Image
Image(file[0])

In [ ]:
# Download trained h5 model
%cd /content/mycar/models
from google.colab import files
files.download(f"{model_name}.h5")

In [ ]:
# Download trained tflite model
%cd /content/mycar/models
from google.colab import files
files.download(f"{model_name}.tflite")

In [ ]:
# OPTIONAL generate movie from training date with predictions of generated h5 model. (If model is of type linear)
%cd /content/mycar
!donkey makemovie --tub ./data/data-0 --model models/{model_name}.h5 --type linear --salient  --start 1 --end 1000 --out prediction.mp4

In [ ]:
# OPTIONAL Download result video
from google.colab import files
files.download("prediction.mp4")